# Loading the processed data

In [1]:
import geopandas as gpd
import numpy as np

In [2]:
import pandas as pd
from shapely.geometry import LineString, MultiPoint, Point
from shapely.ops import nearest_points
from utils import add_release_penalty, map_shapely_objects, vrp_solver, generate_readable_solution, set_up_map

In [3]:
road_crossed = np.load('data/road_crossed.pickle', allow_pickle=True)
water_crossed = np.load('data/water_crossed.pickle', allow_pickle=True)
euclidean_dist = np.load('data/distance.pickle', allow_pickle=True)

In [4]:
# filename = "data/block_with_geoloc.csv" # Old data
filename = "data/NSE_release_with_latlong.csv" # New data

In [5]:
df = pd.read_csv(filename)
num_blocks = len(df)
longlat = list(zip(df["long"], df["lat"]))
# Converting it to geopandas
geometry = [Point(xy) for xy in longlat]
crs = {"init": "epsg:4326"}
gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry).to_crs(epsg=3414)

Getting the centriod from the points

In [6]:
pts = MultiPoint([row['geometry'] for _, row in gdf.iterrows()])
near_cent = nearest_points(pts.centroid, pts)
centriod = gdf.distance(near_cent[0]).idxmin()

## Generating the time/distance matrix
Adding in penalty for road crossing. 

In the future can modify the release_penalty as well

In [12]:
penalty_road = 300 # Time penalty for road crossing
penalty_water = 450 # time penalty for crossing water body
penalty_release = 240 # time needed for each release
depot = centriod

In [31]:
walking_speed = 90  # m/min
euclidean_time_matrix = (euclidean_dist / walking_speed) * 60
euclid_matrix = euclidean_dist
time_matrix = (
    add_release_penalty(euclidean_time_matrix, penalty=penalty_release)
    + road_crossed * penalty_road + water_crossed * penalty_water
)
dist_matrix = euclidean_dist

# Plotting the routes

In [32]:
current_time_matrix = time_matrix
current_distance_matrix = euclidean_dist

In [38]:
%%time
"""Solve the CVRP problem."""
# Instantiate the data problem.
"""Stores the data for the problem."""
data = {}
data["distance_matrix"] = current_time_matrix
data["num_vehicles"] = 6
data["depot"] = depot

solution, manager, routing = vrp_solver(data)

# Print solution on console.
data["time_matrix"] = current_time_matrix
data["dist_matrix"] = current_distance_matrix
data["df"] = df
data["euclid_matrix"] = euclidean_dist
if solution:
    full_route = generate_readable_solution(
        data,
        manager,
        routing,
        solution,
    )

Route for vehicle 0:
Blk 240 -> Blk 238 -> Blk 239 -> Blk 237 -> Blk 236 -> Blk 235 -> Blk 233 -> Blk 234 -> Blk 232 -> Blk 227 -> Blk 225 -> Blk 296 -> Blk 297 -> Blk 298 -> Blk 299 -> Blk 224 -> Blk 223 -> Blk 226 -> Blk 216 -> Blk 213 -> Blk 212 -> Blk 211 -> Blk 311
Walking distance of the route: 2064.5020870907083m
Walking time of the route: 130.93891207878568mins
Euclidean distance of the route: 2064.5020870907083m
Numbers of block: 23
Route for vehicle 1:
Blk 306 -> Blk 305 -> Blk 210 -> Blk 209 -> Blk 207 -> Blk 206 -> Blk 214 -> Blk 217 -> Blk 218 -> Blk 219 -> Blk 221 -> Blk 222 -> Blk 220 -> Blk 215 -> Blk 204 -> Blk 205 -> Blk 203 -> Blk 202 -> Blk 208 -> Blk 201 -> Blk 302 -> Blk 303 -> Blk 330 -> Blk 314 -> Blk 310 -> Blk 311
Walking distance of the route: 2342.9044835219524m
Walking time of the route: 136.03227203913283mins
Euclidean distance of the route: 2342.9044835219524m
Numbers of block: 26
Route for vehicle 2:
Blk 309 -> Blk 242 -> Blk 243 -> Blk 256 -> Blk 257 ->

In [39]:
mapa = set_up_map(data, manager, routing, solution)
mapa

# Arbitrary start and end point

In [11]:
current_time_matrix = np.zeros((num_blocks + 1, num_blocks + 1))
current_time_matrix[:num_blocks, :num_blocks] = (
    add_release_penalty(euclidean_time_matrix, penalty=penalty_release)
    + road_crossed * penalty_road + water_crossed * penalty_water
)
current_distance_matrix = np.zeros((num_blocks + 1, num_blocks + 1))
current_distance_matrix[:num_blocks, :num_blocks] = euclidean_dist
current_euclid_matrix = np.zeros((num_blocks + 1, num_blocks + 1))
current_euclid_matrix[:num_blocks, :num_blocks] = euclidean_dist
df_append = df.copy()
df_append = df_append.append(
    {"Block": "Dummy", "lat": None, "long": None}, ignore_index=True
)

In [12]:
"""Solve the CVRP problem."""
# Instantiate the data problem.
"""Stores the data for the problem."""
data = {}
data["distance_matrix"] = current_time_matrix
data["num_vehicles"] = 7
data["depot"] = num_blocks

# Create the routing index manager.
solution, manager, routing = vrp_solver(data)
data["time_matrix"] = current_time_matrix
data["dist_matrix"] = current_distance_matrix
data["df"] = df_append
data["euclid_matrix"] = current_euclid_matrix
# Print solution on console.
if solution:
    full_route = generate_readable_solution(
        data,
        manager,
        routing,
        solution,
    )

mapa = set_up_map(data, manager, routing, solution)
display(mapa)

Route for vehicle 0:
Blk 362 -> Blk 344 -> Blk 364 -> Blk 365 -> Blk 363 -> Blk 366 -> Blk 361 -> Blk 360 -> Blk 359 -> Blk 358 -> Blk 357 -> Blk 352 -> Blk 353 -> Blk 354 -> Blk 356 -> Blk 355 -> Blk 350 -> Blk 355A -> Blk 350A -> Blk 351 -> Blk 349 -> Blk 348D -> Blk Dummy
Walking distance of the route: 1402.7385907649348m
Walking time of the route: 120.58598434183259mins
Euclidean distance of the route: 1402.7385907649348m
Numbers of block: 23
Route for vehicle 1:
Blk 367 -> Blk 342B -> Blk 342A -> Blk 342C -> Blk 343 -> Blk 345 -> Blk 346 -> Blk 347A -> Blk 347B -> Blk 348C -> Blk 348B -> Blk 348A -> Blk 324 -> Blk 325 -> Blk 327 -> Blk 329 -> Blk 315B -> Blk 315A -> Blk 315C -> Blk Dummy
Walking distance of the route: 1485.7461298828343m
Walking time of the route: 121.50829033203148mins
Euclidean distance of the route: 1485.7461298828343m
Numbers of block: 20
Route for vehicle 2:
Blk 241 -> Blk 240 -> Blk 238 -> Blk 239 -> Blk 237 -> Blk 236 -> Blk 235 -> Blk 233 -> Blk 234 -> Blk

In [13]:
mapa = set_up_map(data, manager, routing, solution)
display(mapa)

# Looping to minimise distance travelled for all

Current method only optimise the longest travelling time and not of the others, so looping to minimise the distance should give better results. 

*Unfortunately this is not the case.*

In [ ]:
current_time_matrix = time_matrix_10min

In [ ]:
"""Solve the CVRP problem."""
# Instantiate the data problem.
"""Stores the data for the problem."""
data = {}
data["distance_matrix"] = current_time_matrix
data["num_vehicles"] = 6
data["depot"] = 37
df_tmp = df.copy()

while data["num_vehicles"] > 2:
    # Create the routing index manager.
    solution, manager, routing = vrp_solver(data)

    # Print solution on console.
    if solution:
        data["df"] = df_tmp
        data["time_matrix"] = current_time_matrix
        print("SOLUTIONS\n")
        full_route = generate_readable_solution(data, manager, routing, solution)
        max_route_id = np.argmax(full_route["time"])
        print("\nMax\n")
        print(full_route["route_print"][max_route_id])
        max_route = full_route["route"][max_route_id][1:-1]
        df_tmp = df_tmp.drop(max_route)
        df_tmp = df_tmp.reset_index(drop=True)
        current_time_matrix = np.delete(current_time_matrix, max_route, 0)
        current_time_matrix = np.delete(current_time_matrix, max_route, 1)
        data["distance_matrix"] = current_time_matrix
        data["num_vehicles"] = data["num_vehicles"] - 1
        data["depot"] = data["depot"] - sum([(i < data["depot"]) for i in max_route])
    else:
        break